In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/krise/Documents/GitHub/private/tidier4ds"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using Tidier

In [3]:
using PalmerPenguins
penguins = DataFrame(PalmerPenguins.load())

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.5,17.4,186,3800,female
3,Adelie,Torgersen,40.3,18.0,195,3250,female
4,Adelie,Torgersen,missing,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450,female
6,Adelie,Torgersen,39.3,20.6,190,3650,male
7,Adelie,Torgersen,38.9,17.8,181,3625,female
8,Adelie,Torgersen,39.2,19.6,195,4675,male
9,Adelie,Torgersen,34.1,18.1,193,3475,missing


In [4]:
@chain penguins begin
    @filter(bill_length_mm > 50)
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Gentoo,Biscoe,50.2,14.3,218,5700,male
2,Gentoo,Biscoe,59.6,17.0,230,6050,male
3,Gentoo,Biscoe,50.5,15.9,222,5550,male
4,Gentoo,Biscoe,50.5,15.9,225,5400,male
5,Gentoo,Biscoe,50.1,15.0,225,5000,male
6,Gentoo,Biscoe,50.4,15.3,224,5550,male
7,Gentoo,Biscoe,54.3,15.7,231,5650,male
8,Gentoo,Biscoe,50.7,15.0,223,5550,male
9,Gentoo,Biscoe,51.1,16.3,220,6000,male


In [5]:
@chain penguins begin
    @filter(island == "Biscoe" && sex == "male") 
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Biscoe,37.7,18.7,180,3600,male
2,Adelie,Biscoe,38.2,18.1,185,3950,male
3,Adelie,Biscoe,38.8,17.2,180,3800,male
4,Adelie,Biscoe,40.6,18.6,183,3550,male
5,Adelie,Biscoe,40.5,18.9,180,3950,male
6,Adelie,Biscoe,40.1,18.9,188,4300,male
7,Adelie,Biscoe,42.0,19.5,200,4050,male
8,Adelie,Biscoe,41.4,18.6,191,3700,male
9,Adelie,Biscoe,40.6,18.8,193,3800,male


In [6]:
@chain penguins begin
    @filter(island == "Biscoe" || sex == "male") 
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.3,20.6,190,3650,male
3,Adelie,Torgersen,39.2,19.6,195,4675,male
4,Adelie,Torgersen,38.6,21.2,191,3800,male
5,Adelie,Torgersen,34.6,21.1,198,4400,male
6,Adelie,Torgersen,42.5,20.7,197,4500,male
7,Adelie,Torgersen,46.0,21.5,194,4200,male
8,Adelie,Biscoe,37.8,18.3,174,3400,female
9,Adelie,Biscoe,37.7,18.7,180,3600,male


In [7]:
@chain penguins begin
    @filter(island in ("Biscoe", "Dream"))
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Biscoe,37.8,18.3,174,3400,female
2,Adelie,Biscoe,37.7,18.7,180,3600,male
3,Adelie,Biscoe,35.9,19.2,189,3800,female
4,Adelie,Biscoe,38.2,18.1,185,3950,male
5,Adelie,Biscoe,38.8,17.2,180,3800,male
6,Adelie,Biscoe,35.3,18.9,187,3800,female
7,Adelie,Biscoe,40.6,18.6,183,3550,male
8,Adelie,Biscoe,40.5,17.9,187,3200,female
9,Adelie,Biscoe,37.9,18.6,172,3150,female


In [8]:
penguins_male = @chain penguins begin
    @filter(sex == "male") 
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.3,20.6,190,3650,male
3,Adelie,Torgersen,39.2,19.6,195,4675,male
4,Adelie,Torgersen,38.6,21.2,191,3800,male
5,Adelie,Torgersen,34.6,21.1,198,4400,male
6,Adelie,Torgersen,42.5,20.7,197,4500,male
7,Adelie,Torgersen,46.0,21.5,194,4200,male
8,Adelie,Biscoe,37.7,18.7,180,3600,male
9,Adelie,Biscoe,38.2,18.1,185,3950,male


In [9]:
@chain penguins begin
    @arrange(sex, body_mass_g, flipper_length_mm)
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Chinstrap,Dream,46.9,16.6,192,2700,female
2,Adelie,Biscoe,36.5,16.6,181,2850,female
3,Adelie,Biscoe,36.4,17.1,184,2850,female
4,Adelie,Dream,33.1,16.1,178,2900,female
5,Adelie,Biscoe,34.5,18.1,187,2900,female
6,Chinstrap,Dream,43.2,16.6,187,2900,female
7,Adelie,Torgersen,38.6,17.0,188,2900,female
8,Adelie,Biscoe,37.9,18.6,193,2925,female
9,Adelie,Dream,37.0,16.9,185,3000,female


In [10]:
@chain penguins begin
    @arrange(desc(sex), body_mass_g, flipper_length_mm)
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Dream,37.5,18.9,179,2975,missing
2,Adelie,Torgersen,37.8,17.1,186,3300,missing
3,Adelie,Torgersen,34.1,18.1,193,3475,missing
4,Adelie,Torgersen,37.8,17.3,180,3700,missing
5,Gentoo,Biscoe,44.5,14.3,216,4100,missing
6,Adelie,Torgersen,42.0,20.2,190,4250,missing
7,Gentoo,Biscoe,46.2,14.4,214,4650,missing
8,Gentoo,Biscoe,47.3,13.8,216,4725,missing
9,Gentoo,Biscoe,44.5,15.7,217,4875,missing


In [11]:
@chain penguins begin
    @distinct()
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.5,17.4,186,3800,female
3,Adelie,Torgersen,40.3,18.0,195,3250,female
4,Adelie,Torgersen,missing,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450,female
6,Adelie,Torgersen,39.3,20.6,190,3650,male
7,Adelie,Torgersen,38.9,17.8,181,3625,female
8,Adelie,Torgersen,39.2,19.6,195,4675,male
9,Adelie,Torgersen,34.1,18.1,193,3475,missing


In [12]:
@chain penguins begin
    @distinct(species, island)
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Biscoe,37.8,18.3,174,3400,female
3,Adelie,Dream,39.5,16.7,178,3250,female
4,Gentoo,Biscoe,46.1,13.2,211,4500,female
5,Chinstrap,Dream,46.5,17.9,192,3500,female


In [13]:
@chain penguins begin
    @distinct(species, island)
    @select(species, island)
end

Row,species,island
,String15,String15
1,Adelie,Torgersen
2,Adelie,Biscoe
3,Adelie,Dream
4,Gentoo,Biscoe
5,Chinstrap,Dream


In [14]:
@chain penguins begin
    @mutate(
        bill_ratio_mm = bill_length_mm / bill_depth_mm
    )
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,bill_ratio_mm
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7,Float64?
1,Adelie,Torgersen,39.1,18.7,181,3750,male,2.09091
2,Adelie,Torgersen,39.5,17.4,186,3800,female,2.27011
3,Adelie,Torgersen,40.3,18.0,195,3250,female,2.23889
4,Adelie,Torgersen,missing,missing,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450,female,1.90155
6,Adelie,Torgersen,39.3,20.6,190,3650,male,1.90777
7,Adelie,Torgersen,38.9,17.8,181,3625,female,2.18539
8,Adelie,Torgersen,39.2,19.6,195,4675,male,2.0
9,Adelie,Torgersen,34.1,18.1,193,3475,missing,1.88398


In [15]:
@chain penguins begin
    @mutate(
        bill_ratio_mm = bill_length_mm / bill_depth_mm
    )
    @mutate(
        bill_ratio_mm_2 = bill_ratio_mm * 2
    )
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,bill_ratio_mm,bill_ratio_mm_2
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7,Float64?,Float64?
1,Adelie,Torgersen,39.1,18.7,181,3750,male,2.09091,4.18182
2,Adelie,Torgersen,39.5,17.4,186,3800,female,2.27011,4.54023
3,Adelie,Torgersen,40.3,18.0,195,3250,female,2.23889,4.47778
4,Adelie,Torgersen,missing,missing,missing,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450,female,1.90155,3.80311
6,Adelie,Torgersen,39.3,20.6,190,3650,male,1.90777,3.81553
7,Adelie,Torgersen,38.9,17.8,181,3625,female,2.18539,4.37079
8,Adelie,Torgersen,39.2,19.6,195,4675,male,2.0,4.0
9,Adelie,Torgersen,34.1,18.1,193,3475,missing,1.88398,3.76796


In [16]:
@chain penguins begin
    @select(species, island, bill_length_mm)
end

Row,species,island,bill_length_mm
,String15,String15,Float64?
1,Adelie,Torgersen,39.1
2,Adelie,Torgersen,39.5
3,Adelie,Torgersen,40.3
4,Adelie,Torgersen,missing
5,Adelie,Torgersen,36.7
6,Adelie,Torgersen,39.3
7,Adelie,Torgersen,38.9
8,Adelie,Torgersen,39.2
9,Adelie,Torgersen,34.1


In [17]:
@chain penguins begin
    @select(species:bill_length_mm)
end

Row,species,island,bill_length_mm
,String15,String15,Float64?
1,Adelie,Torgersen,39.1
2,Adelie,Torgersen,39.5
3,Adelie,Torgersen,40.3
4,Adelie,Torgersen,missing
5,Adelie,Torgersen,36.7
6,Adelie,Torgersen,39.3
7,Adelie,Torgersen,38.9
8,Adelie,Torgersen,39.2
9,Adelie,Torgersen,34.1


In [18]:
@chain penguins begin
    @select(!bill_length_mm)
end

Row,species,island,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,18.7,181,3750,male
2,Adelie,Torgersen,17.4,186,3800,female
3,Adelie,Torgersen,18.0,195,3250,female
4,Adelie,Torgersen,missing,missing,missing,missing
5,Adelie,Torgersen,19.3,193,3450,female
6,Adelie,Torgersen,20.6,190,3650,male
7,Adelie,Torgersen,17.8,181,3625,female
8,Adelie,Torgersen,19.6,195,4675,male
9,Adelie,Torgersen,18.1,193,3475,missing


In [19]:
#| eval: false
# TODO: where() is not defined yet!
@chain penguins begin
    @select(where(is_string))
end

In [20]:
@chain penguins begin
    @select(bill_depth = bill_depth_mm)
end

Row,bill_depth
,Float64?
1,18.7
2,17.4
3,18.0
4,missing
5,19.3
6,20.6
7,17.8
8,19.6
9,18.1


In [21]:
@chain penguins begin
    @rename(bill_depth = bill_depth_mm,
            bill_length = bill_length_mm)
end

Row,species,island,bill_length,bill_depth,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.5,17.4,186,3800,female
3,Adelie,Torgersen,40.3,18.0,195,3250,female
4,Adelie,Torgersen,missing,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450,female
6,Adelie,Torgersen,39.3,20.6,190,3650,male
7,Adelie,Torgersen,38.9,17.8,181,3625,female
8,Adelie,Torgersen,39.2,19.6,195,4675,male
9,Adelie,Torgersen,34.1,18.1,193,3475,missing


In [22]:
# TODO: relocate() not defined yet!

In [23]:
@chain penguins begin
    @filter(island in ("Biscoe", "Dream"))
    @mutate(bill_ratio_mm = bill_length_mm / bill_depth_mm) 
    @select(bill_ratio_mm, sex, species:island)
    @arrange(bill_ratio_mm)
end

Row,bill_ratio_mm,sex,species,island
,Float64?,String7,String15,String15
1,1.85782,male,Adelie,Dream
2,1.86154,male,Adelie,Dream
3,1.86772,female,Adelie,Biscoe
4,1.86979,female,Adelie,Biscoe
5,1.89,male,Adelie,Biscoe
6,1.90608,female,Adelie,Biscoe
7,1.91,male,Adelie,Biscoe
8,1.91304,female,Adelie,Biscoe
9,1.94,male,Adelie,Dream


In [24]:
@chain penguins begin
    @group_by(sex)
end

Row,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,String15,String15,Float64?,Float64?,Int64?,Int64?,String7
1,Adelie,Torgersen,39.1,18.7,181,3750,male
2,Adelie,Torgersen,39.3,20.6,190,3650,male
3,Adelie,Torgersen,39.2,19.6,195,4675,male
4,Adelie,Torgersen,38.6,21.2,191,3800,male
5,Adelie,Torgersen,34.6,21.1,198,4400,male
6,Adelie,Torgersen,42.5,20.7,197,4500,male
7,Adelie,Torgersen,46.0,21.5,194,4200,male
8,Adelie,Biscoe,37.7,18.7,180,3600,male
9,Adelie,Biscoe,38.2,18.1,185,3950,male


In [25]:
@chain penguins begin
    @group_by(island)
    @summarize(bill_length_mean = mean(bill_length_mm),
               n = n())
end

Row,island,bill_length_mean,n
,String15,Float64?,Int64
1,Torgersen,missing,52
2,Biscoe,missing,168
3,Dream,44.1677,124


In [26]:
@chain penguins begin
    @group_by(island)
    @summarize(bill_length_mean = mean(skipmissing(bill_length_mm)),
               n = n())
end

Row,island,bill_length_mean,n
,String15,Float64,Int64
1,Torgersen,38.951,52
2,Biscoe,45.2575,168
3,Dream,44.1677,124
